## Import Libraries

In [116]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# import data handling tools
import pandas as pd
import numpy as np
# import data visualization tools
import matplotlib.pyplot as plt
import seaborn as sns

## Load Data

### News

In [117]:
# Reddit Data
df_Crypto_Currency_News = pd.read_csv('../Load/Reddit/comments/Crypto_Currency_News_comments.csv')
df_CryptoCurrencies = pd.read_csv('../Load/Reddit/comments/CryptoCurrencies_comments.csv')
df_CryptoCurrency = pd.read_csv('../Load/Reddit/comments/CryptoCurrency_comments.csv')
df_Cryptomarkets = pd.read_csv('../Load/Reddit/comments/Cryptomarkets_comments.csv')
df_eth = pd.read_csv('../Load/Reddit/comments/eth_comments.csv')
df_ethfinance = pd.read_csv('../Load/Reddit/comments/ethfinance_comments.csv')
df_ethtrader = pd.read_csv('../Load/Reddit/comments/ethtrader_comments.csv')

df_reddit = [df_Crypto_Currency_News, df_CryptoCurrencies, df_CryptoCurrency, df_Cryptomarkets, df_eth, df_ethfinance, df_ethtrader]

### Financial Data

In [261]:
# Yahoo Data
df_yahoo = pd.read_csv('../Load/datasets/Yahoo/ETH-USD.csv', delimiter=';')
# Binanace Data
df_BNB_ETH = pd.read_csv('../Load/datasets/Binance/BNB-ETH-klines.csv', delimiter=';')
df_ETH_BTC = pd.read_csv('../Load/datasets/Binance/ETH-BTC-klines.csv', delimiter=';')
df_ETH_USDT = pd.read_csv('../Load/datasets/Binance/ETH-USDT-klines.csv', delimiter=';')
df_XRP_ETH = pd.read_csv('../Load/datasets/Binance/XRP-ETH-klines.csv', delimiter=';')

df_binance = [df_ETH_USDT, df_ETH_BTC, df_BNB_ETH, df_XRP_ETH]

## Transform Dataframes

### Reddit

In [119]:
for df in df_reddit:
    # Drop columns we don't need
    df.drop(['id', 'num_comments', 'permalink', 'score', 'subreddit'], axis=1, inplace=True)
    # Change created_utc format to day/month/year
    df['date'] = pd.to_datetime(df['created_utc'])
    df['date'] = df['date'].dt.strftime('%#d/%#m/%Y')
    # Join title and comments into one column
    # if a comment is empty, replace it with an empty string
    df['title'] = df['title'].astype(str)
    df['comment_1'].fillna('', inplace=True)
    df['comment_1'] = df['comment_1'].astype(str)
    df['comment_2'].fillna('', inplace=True)
    df['comment_2'] = df['comment_2'].astype(str)
    df['comment_3'].fillna('', inplace=True)
    df['comment_3'] = df['comment_3'].astype(str)
    df['text'] = df['title'] + '. ' + df['comment_1'] + '. ' + df['comment_2'] + '. ' + df['comment_3']
    # Drop columns
    df.drop(['created_utc','title', 'comment_1', 'comment_2', 'comment_3'], axis=1, inplace=True)
    
pd.set_option('display.max_rows', None)
df_Crypto_Currency_News.head(11)

,date,text
0,1/10/2019,Divi Project Update September 2019 Month in Re...
1,1/10/2019,Interview with the CEO of DIVI Geoff McCabe by...
2,1/10/2019,Is Wall Street Finally Learning Not To Panic O...
3,1/10/2019,"In China, a large mining farm burned down. Dam..."
4,1/10/2019,Bitcoin Bull Run's 'Likely Target' is $200-300...
5,1/10/2019,"Vaultoro Integrates Dash, Relaunches Gold-to-C..."
6,1/10/2019,Flixxo's latest article on LATAM Bonds. . .
7,1/10/2019,Block.One agrees to pay SEC $24 million over u...
8,1/10/2019,Altcoin News: Shady CoinExchange.io Is Closing...
9,1/10/2019,Which cryptocurrency do you think will win the...


Make a list of all the Dates in our Range

In [120]:
from datetime import datetime,timedelta

start_date = datetime.strptime("1/10/2019","%d/%m/%Y")
end_date = datetime.strptime("30/9/2022","%d/%m/%Y")

# This will create a list with complete dates
completeDates = [start_date + timedelta(days=x) for x in range(0,(end_date-start_date ).days + 1)]
# reformat completeDates from 2019-10-01 00:00:00 to 1/10/2019
completeDates = [date.strftime('%#d/%#m/%Y') for date in completeDates]

Reshape Reddit Dataframes to have one row per Date, with a column for each text

In [121]:
# create dummy dataframes for the 7 reddit dataframes
columns = ['date', 'text_1', 'text_2', 'text_3', 'text_4', 'text_5', 'text_6', 'text_7', 'text_8', 'text_9', 'text_10']
df_Crypto_Currency_News_f = pd.DataFrame(columns=columns)
df_CryptoCurrencies_f = pd.DataFrame(columns=columns)
df_CryptoCurrency_f = pd.DataFrame(columns=columns)
df_Cryptomarkets_f = pd.DataFrame(columns=columns)
df_eth_f = pd.DataFrame(columns=columns)
df_ethfinance_f = pd.DataFrame(columns=columns)
df_ethtrader_f = pd.DataFrame(columns=columns)

# create list of dataframes
df_reddit_f = [df_Crypto_Currency_News_f, df_CryptoCurrencies_f, df_CryptoCurrency_f, df_Cryptomarkets_f, df_eth_f, df_ethfinance_f, df_ethtrader_f]

# get dates from completeDates list
for df in df_reddit_f:
    df['date'] = completeDates

df_Crypto_Currency_News_f.head(3)

,date,text_1,text_2,text_3,text_4,text_5,text_6,text_7,text_8,text_9,text_10
0,1/10/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2/10/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3/10/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
# Should take around 40 seconds
for i, df in enumerate(df_reddit):
    for day in completeDates:
        df_day = df.loc[df['date'] == day].reset_index(drop=True)
        # get the number of rows
        df_day_len = len(df_day)
        # if there are rows
        if df_day_len > 0:
            for j in range(0, df_day_len):
                text = df_day['text'][j]
                column = 'text_' + str(j+1)
                index = df_reddit_f[i].index[df_reddit_f[i]['date'] == day].values[0]
                df_reddit_f[i].at[index, column] = text

In [123]:
# For the resulting dataframes, set date as index
for df in df_reddit_f:
    df.set_index('date', inplace=True)

In [142]:
# verify that all dataframes have all the dates
for df in df_reddit_f:
    myDates = df.index.tolist()
    myDates = [d.split()[0] for d in myDates]
    missingDates = [d for d in completeDates if d not in myDates]
    print(len(missingDates))


0
0
0
0
0
0
0


In [144]:
# Lets see the final results of one dataframe
df_Crypto_Currency_News_f.head(2)

,text_1,text_2,text_3,text_4,text_5,text_6,text_7,text_8,text_9,text_10
date,,,,,,,,,,
1/10/2019,Divi Project Update September 2019 Month in Re...,Interview with the CEO of DIVI Geoff McCabe by...,Is Wall Street Finally Learning Not To Panic O...,"In China, a large mining farm burned down. Dam...",Bitcoin Bull Run's 'Likely Target' is $200-300...,"Vaultoro Integrates Dash, Relaunches Gold-to-C...",Flixxo's latest article on LATAM Bonds. . .,Block.One agrees to pay SEC $24 million over u...,Altcoin News: Shady CoinExchange.io Is Closing...,Which cryptocurrency do you think will win the...
2/10/2019,Popular Ethereum App Collapses: How Many Other...,"Top German Bank Predicts Bitcoin to Hit $90,00...",UTRUST Integrates Dash Enabling Cryptocurrency...,Brazilian Cryptocurrency Exchange Cointrade.cx...,Samsung-Backed Blocko Launches in UAE After Du...,Speaker of the House Challenger Bacelar has a ...,Altcoin News: A Large German Bank Predicts a R...,Trustee of Mt.Gox Is Interested in the Case of...,HONG KONG: Why Bitcoin volumes have hit a reco...,US State of Ohio Suspends Service for Paying T...


In [140]:
df_Crypto_Currency_News_f.tail(2)

,text_1,text_2,text_3,text_4,text_5,text_6,text_7,text_8,text_9,text_10
date,,,,,,,,,,
29/9/2022,Upvote For Upvote. Can Bitcoin break the resis...,Audius (AUDIO) Decentralized Music Platform Re...,"Smoke Session! Comment ""puff"" for your Stellar...",Binance Launches Training Program for Governme...,Ribbon Finance has launches ETH-Based Crypto O...,ECB Advisor Defends Amazonâ€™s Role in Digital...,Brazilâ€™s Securities Market Regulator Targets...,win free virtual land on Metaverse. . .,ChainPort adds support for Aurora EVM. The exp...,Project for cross-chain bitcoin transfers invo...
30/9/2022,Crypto Miners are Ready for the Future. . .,OKX Reddit Giveaway | You're in ?. . .,MicroStrategy Looks to Hire Software Engineer ...,MetaMask Launches New All-In-One Digital Asset...,How to make MONEY online for FREE in 2022?????...,Crypto Underworld â€“ Where Crypto Profits are...,Unique Girls - NFT. . .,TOP METAVERSE DEVELOPMENT USE CASES PREFERRED ...,ONI is available on Guarda wallet. You can sto...,PolkaStation. . .


In [145]:
# check for empty values
df_Crypto_Currency_News_f.isnull().sum()

text_1     131
text_2     131
text_3     131
text_4     132
text_5     133
text_6     133
text_7     133
text_8     134
text_9     134
text_10    135
dtype: int64

### Yahoo & Binance

In [180]:
# No changes needded for Yahoo data
df_yahoo.head()

,date,adj_close,volume
0,1/10/2019,177.340424,7676276225
1,2/10/2019,180.710510,6335595250
2,3/10/2019,175.199341,6381403725
3,4/10/2019,176.985001,6248928449
4,5/10/2019,176.351517,5837211771


In [262]:
for df in df_binance:
    # Drop columns we don't need
    df.drop(['quote_asset_volume', 'taker_buy_quote_asset_volume'], axis=1, inplace=True)
    # For df_ETH_USDT we also drop close, as we have the equivalent in df_yahoo
    if df is df_ETH_USDT:
        df.drop(['close'], axis=1, inplace=True)

# Rename columns
df_ETH_USDT.rename(columns={'num_trades':'num_trades_USDT'}, inplace=True)

df_ETH_BTC.rename(columns={'close':'close_BTC'}, inplace=True)
df_ETH_BTC.rename(columns={'num_trades':'num_trades_BTC'}, inplace=True)

df_BNB_ETH.rename(columns={'close':'close_BNB'}, inplace=True)
df_BNB_ETH.rename(columns={'num_trades':'num_trades_BNB'}, inplace=True)

df_XRP_ETH.rename(columns={'close':'close_XRP'}, inplace=True)
df_XRP_ETH.rename(columns={'num_trades':'num_trades_XRP'}, inplace=True)
# Fix XRP_ETH date format from 01/09/2019 to 1/9/2019
df_XRP_ETH['date'] = df_XRP_ETH['date'].str.replace(r'\b0(\d{1})', r'\1')

df_XRP_ETH.head()

,date,close_BTC,num_trades_BTC
0,1/10/2019,0.021182,133240
1,2/10/2019,0.021566,92471
2,3/10/2019,0.021241,87748
3,4/10/2019,0.021580,87725
4,5/10/2019,0.021682,93049


In [264]:
# Join yahoo and binance dataframes on equal dates
df_finance = pd.merge(df_yahoo, df_ETH_USDT, on='date', how='left')
df_finance = pd.merge(df_finance, df_ETH_BTC, on='date', how='left')
df_finance = pd.merge(df_finance, df_BNB_ETH, on='date', how='left')
df_finance = pd.merge(df_finance, df_XRP_ETH, on='date', how='left')

# in case any dates duplicate, we drop them
df_finance.drop_duplicates(subset='date', keep='first', inplace=True)

# set date as index
df_finance.set_index('date', inplace=True)
df_finance.index = pd.to_datetime(df_finance.index, format="%d/%m/%Y")

df_finance.head()

,adj_close,volume,num_trades_USDT,close_BTC,num_trades_BTC,close_BNB,num_trades_BNB,close_XRP,num_trades_XRP
date,,,,,,,,,
2019-10-01,177.340424,7676276225,168492,0.021182,133240,0.090262,11802,0.001410,5946.0
2019-10-02,180.710510,6335595250,96660,0.021566,92471,0.088028,7465,0.001401,5689.0
2019-10-03,175.199341,6381403725,101114,0.021241,87748,0.089504,7160,0.001413,4310.0
2019-10-04,176.985001,6248928449,89655,0.021580,87725,0.089029,6246,0.001439,4729.0
2019-10-05,176.351517,5837211771,90341,0.021682,93049,0.089143,5373,0.001437,4356.0


In [266]:
# verify that all dates are present
myDates = df_finance.index.tolist()
myDates = [d.strftime('%#d/%#m/%Y') for d in myDates]
missingDates = [d for d in completeDates if d not in myDates]
print(len(missingDates))

0


#### Explaining vairables
**closes**
- adj_close and close_BTC is the price of ETH relative to USD and BTC respectively. If adj_close = 1328 and close_BTC = 0.068, then 1 ETH is worth 1328 USD and 0.068 BTC.
- close_BNB and close_XRP is the price of BNB and XRP in ETH, the other way around. If close_BNB = 0.021 close_XRP = 0.00036, then 1 BNB is worth 0.021 ETH and 1 XRP is worth 0.00036 ETH.

**volume**
- Shows how much was traded on each exchange where ETH-USD is listed. It is an indicator of its popularity as well as market sentiment.

**num_trades**
- Number of trades is the number of trades that happened that day between ETH and the other coin in Binance

In [188]:
# check for empty values
df_finance.isnull().sum()

adj_close           0
volume              0
num_trades_USDT     0
close_BTC           0
num_trades_BTC      0
close_BNB           0
num_trades_BNB      0
close_XRP          61
num_trades_XRP     61
dtype: int64

In [267]:
df_finance['close_XRP'] = df_finance['close_XRP'].interpolate(method='time')
df_finance['num_trades_XRP'] = df_finance['num_trades_XRP'].interpolate(method='time')

In [268]:
df_finance.isnull().sum()

adj_close          0
volume             0
num_trades_USDT    0
close_BTC          0
num_trades_BTC     0
close_BNB          0
num_trades_BNB     0
close_XRP          0
num_trades_XRP     0
dtype: int64

In [269]:
# fix index format from 2019-10-01 to 1/10/2019
df_finance.index = df_finance.index.strftime('%#d/%#m/%Y')

In [270]:
df_finance.head()

,adj_close,volume,num_trades_USDT,close_BTC,num_trades_BTC,close_BNB,num_trades_BNB,close_XRP,num_trades_XRP
date,,,,,,,,,
1/10/2019,177.340424,7676276225,168492,0.021182,133240,0.090262,11802,0.001410,5946.0
2/10/2019,180.710510,6335595250,96660,0.021566,92471,0.088028,7465,0.001401,5689.0
3/10/2019,175.199341,6381403725,101114,0.021241,87748,0.089504,7160,0.001413,4310.0
4/10/2019,176.985001,6248928449,89655,0.021580,87725,0.089029,6246,0.001439,4729.0
5/10/2019,176.351517,5837211771,90341,0.021682,93049,0.089143,5373,0.001437,4356.0


In [271]:
df_finance.tail()

,adj_close,volume,num_trades_USDT,close_BTC,num_trades_BTC,close_BNB,num_trades_BNB,close_XRP,num_trades_XRP
date,,,,,,,,,
26/9/2022,1335.320190,16034549271,934159,0.069505,275490,0.2066,23684,0.000350,15980.0
27/9/2022,1330.127686,17870598937,1055212,0.069609,356483,0.2050,29137,0.000335,14991.0
28/9/2022,1337.410889,18994979566,1075441,0.068884,404236,0.2105,37346,0.000336,13145.0
29/9/2022,1335.652344,13796915736,950476,0.068188,325040,0.2126,29551,0.000365,17534.0
30/9/2022,1327.978638,14250100093,926935,0.068412,345607,0.2143,28865,0.000361,21250.0
